# Data Preparation

- Get a dataset of PDF files of financial reports from a big listed public company
- Pipeline to convert pdf to appropriate data format

## Imports

In [11]:
from pathlib import Path
from PyPDF2 import PdfReader
from typing import List, Tuple
# import camelot
import tabula
import pandas as pd

In [3]:
from haystack.nodes import PreProcessor, PDFToTextConverter
from haystack import Document

C:\Users\KianBoon\anaconda3\envs\temus\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Data

In [4]:
DATA_DIR = Path("../data/")

In [5]:
sample_data = DATA_DIR / "2021_Q3_alphabet_10Q.pdf"

In [6]:
sample_data.exists()

True

## Extract text

### Using PyPDF2
https://pypdf2.readthedocs.io/en/3.0.0/user/extract-text.html

In [6]:
reader = PdfReader(sample_data)

In [9]:
page = reader.pages[4]

In [16]:
print(page.extract_text())

PART I. FINANCIAL INFORMATION
ITEM 1. FINANCIAL STATEMENTS
Alphabet Inc.
CONSOLIDATED BALANCE SHEETS
(in millions, except share amounts which are reflected in thousands, and par value per share amounts)
As of 
December 31, 2020As of
September 30, 2021
(unaudited)
Assets
Current assets:
Cash and cash equivalents $ 26,465 $ 23,719 
Marketable securities  110,229  118,284 
Total cash, cash equivalents, and marketable securities  136,694  142,003 
Accounts receivable, net  30,930  34,047 
Income taxes receivable, net  454  753 
Inventory  728  1,278 
Other current assets  5,490  6,029 
Total current assets  174,296  184,110 
Non-marketable investments  20,703  26,101 
Deferred income taxes  1,084  1,195 
Property and equipment, net  84,749  94,631 
Operating lease assets  12,211  12,918 
Intangible assets, net  1,445  1,549 
Goodwill  21,175  22,623 
Other non-current assets  3,953  4,276 
Total assets $ 319,616 $ 347,403 
Liabilities and Stockholders’ Equity
Current liabilities:
Accounts 

In [22]:
doc = ""
for page in reader.pages:
    doc += page.extract_text() + "\n"

### Using Haystack
https://haystack.deepset.ai/tutorials/08_preprocessing

Requires the installation of [PyMuPDF](https://pymupdf.readthedocs.io/en/latest/index.html)

#### `remove_numeric_tables` = False

In [12]:
converter = PDFToTextConverter(remove_numeric_tables=False, valid_languages=["en"])

In [13]:
%%time
doc_pdf = converter.convert(file_path=sample_data, meta=None)[0]

CPU times: total: 266 ms
Wall time: 18.7 s


In [14]:
print(doc_pdf.content)

UNITED STATES
SECURITIES AND EXCHANGE COMMISSION
Washington, D.C. 20549
________________________________________________________________________________________
FORM 10-Q 
________________________________________________________________________________________
(Mark One)
☒
QUARTERLY REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934
For the quarterly period ended September 30, 2021
OR
☐
TRANSITION REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934
For the transition period from _______ to _______
Commission file number: 001-37580 
________________________________________________________________________________________
Alphabet Inc. 
(Exact name of registrant as specified in its charter)
________________________________________________________________________________________
Delaware
61-1767919
(State or other jurisdiction of incorporation or organization)
(I.R.S. Employer Identification Number)
1600 Amphitheatre Parkway
Mountain Vie

#### `remove_numeric_tables` = True

In [10]:
converter = PDFToTextConverter(remove_numeric_tables=True, valid_languages=["en"])

In [ ]:
%%time
doc_pdf = converter.convert(file_path=sample_data, meta=None)[0]

CPU times: total: 797 ms
Wall time: 21.1 s


In [ ]:
print(doc_pdf.content)

UNITED STATES
SECURITIES AND EXCHANGE COMMISSION
Washington, D.C. 20549
________________________________________________________________________________________
________________________________________________________________________________________
(Mark One)
☒
QUARTERLY REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934
For the quarterly period ended September 30, 2021
OR
☐
TRANSITION REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934
For the transition period from _______ to _______
Commission file number: 001-37580 
________________________________________________________________________________________
Alphabet Inc. 
(Exact name of registrant as specified in its charter)
________________________________________________________________________________________
Delaware
(State or other jurisdiction of incorporation or organization)
(I.R.S. Employer Identification Number)
1600 Amphitheatre Parkway
Mountain View, CA 94043
(Address o

## Extract tables

### Using [Camelot](https://camelot-py.readthedocs.io/en/master/index.html)

In [11]:
tables = camelot.read_pdf(str(sample_data), flavor='stream')

In [16]:
tables

<TableList n=1>

In [17]:
tables[0].df.head()

,0
0,UNITED STATES
1,SECURITIES AND EXCHANGE COMMISSION
2,"Washington, D.C. 20549"
3,____________________________________________________________________________...
4,FORM 10-Q


In [18]:
tables[0].parsing_report

{'accuracy': 100.0, 'whitespace': 0.0, 'order': 1, 'page': 1}

### Using [tabula-py](https://tabula-py.readthedocs.io/en/latest/getting_started.html#installation)

In [6]:
# Read pdf into a list of DataFrame
dfs = tabula.read_pdf(sample_data, pages='all')

In [7]:
len(dfs)

99

In [8]:
dfs[0].head(10)

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,(unaudited)
0,Assets,NaN,NaN,NaN,NaN
1,Current assets:,NaN,NaN,NaN,NaN
2,Cash and cash equivalents,$,"26,465",$,"23,719"
3,Marketable securities,NaN,"110,229",NaN,"118,284"
4,"Total cash, cash equivalents, and marketable securities",NaN,"136,694",NaN,"142,003"
5,"Accounts receivable, net",NaN,"30,930",NaN,"34,047"
6,"Income taxes receivable, net",NaN,454,NaN,753
7,Inventory,NaN,728,NaN,"1,278"
8,Other current assets,NaN,"5,490",NaN,"6,029"
9,Total current assets,NaN,"174,296",NaN,"184,110"


## Preprocessing

In [9]:
# doc = Document(content=doc)

NameError: name 'doc_pdf' is not defined

In [13]:
preprocessor = PreProcessor(
    clean_empty_lines=True,
    clean_whitespace=True,
    split_by="word",
    split_length=100,
    split_respect_sentence_boundary=True,
)

In [14]:
docs_default = preprocessor.process([doc_pdf])

Preprocessing: 100%|███████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  9.69docs/s]


In [15]:
len(docs_default)

233

In [16]:
docs_default[0].content

'UNITED STATES\nSECURITIES AND EXCHANGE COMMISSION\nWashington, D.C. 20549\n________________________________________________________________________________________\n________________________________________________________________________________________\n(Mark One)\n☒\nQUARTERLY REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934\nFor the quarterly period ended September 30, 2021\nOR\n☐\nTRANSITION REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934\nFor the transition period from _______ to _______\nCommission file number: 001-37580\n________________________________________________________________________________________\nAlphabet Inc.\n(Exact name of registrant as specified in its charter)\n________________________________________________________________________________________\nDelaware\n(State or other jurisdiction of incorporation or organization)\n(I.R.S. '

## Modularization

In [13]:
def pdf_to_text_and_tables(dir_path: Path) -> Tuple[List[Document], List[pd.DataFrame]]:
    # text passages, tables are represented as Document objects in Haystack
    converter = PDFToTextConverter(remove_numeric_tables=True, valid_languages=["en"])

    text_results = []
    table_results = []
    for i in dir_path.iterdir():
        doc_pdf = converter.convert(file_path=i, meta=None)[0]
        text_results.append(doc_pdf)
        
        tables = tabula.read_pdf(i, pages='all')
        table_results.extend(tables)
    
    return text_results, table_results

In [14]:
%%time
raw_text, raw_tables = pdf_to_text_and_tables(DATA_DIR)

CPU times: total: 5.08 s
Wall time: 7min 46s


In [15]:
len(raw_text), len(raw_tables)

(12, 854)

In [17]:
raw_text[0]

<Document: {'content': 'UNITED STATES\nSECURITIES AND EXCHANGE COMMISSION\nWashington, D.C. 20549\n________________________________________________________________________________________\n________________________________________________________________________________________\n(Mark One)\n☒\nQUARTERLY REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934\nFor the quarterly period ended March 31, 2020 \nOR\n☐\nTRANSITION REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934\nFor the transition period from _______ to _______\nCommission file number: 001-37580 \n________________________________________________________________________________________\nAlphabet Inc. \n(Exact name of registrant as specified in its charter)\n________________________________________________________________________________________\nDelaware\n(State or other jurisdiction of incorporation or organization)\n(I.R.S. Employer Identification Number)\n1600 Amphitheatre 

In [18]:
raw_tables[0]

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,(unaudited)
0,Assets,NaN,NaN,NaN,NaN
1,Current assets:,NaN,NaN,NaN,NaN
2,Cash and cash equivalents,$,"18,498",$,"19,644"
3,Marketable securities,NaN,"101,177",NaN,"97,585"
4,"Total cash, cash equivalents, and marketable securities",NaN,"119,675",NaN,"117,229"
5,"Accounts receivable, net",NaN,"25,326",NaN,"21,825"
6,"Income taxes receivable, net",NaN,"2,166",NaN,"1,910"
7,Inventory,NaN,999,NaN,889
8,Other current assets,NaN,"4,412",NaN,"5,165"
9,Total current assets,NaN,"152,578",NaN,"147,018"


In [20]:
def preprocess_text_documents(documents: List[Document], params: dict = {}):
    default_params = {
        'clean_empty_lines':True,
        'clean_whitespace':True,
        'split_by':"word",
        'split_length':100,
        'split_respect_sentence_boundary':True,
    }
    params = {**default_params, **params}
    
    preprocessor = PreProcessor(**params)
    results = preprocessor.process(documents)
    
    return results

In [21]:
%%time
processed_text_documents = preprocess_text_documents(raw_text)

Preprocessing: 100%|█████████████████████████████████████████████████████████████████| 12/12 [00:00<00:00, 12.07docs/s]

CPU times: total: 984 ms
Wall time: 1 s


In [23]:
len(processed_text_documents)

4124